In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'urban_centers', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\ProgramData\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by Regional Geography
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
df.rename(columns={'name': 'Regional Growth Center'}, inplace=True)
df['Regional Growth Center'] = df['Regional Growth Center'].fillna('Not in RGC')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,276,"3,789","10,455","38,138"
Bellevue,"1,230","7,762","19,377","65,156"
Bothell Canyon Park,"1,615","11,815","18,925","130,687"
Bremerton,73,635,"2,551","14,999"
Burien,256,"1,740","8,853","66,070"
Everett,0,"8,085","19,273","67,768"
Federal Way,66,"1,025","3,760","20,357"
Greater Downtown Kirkland,685,"7,976","16,905","120,239"
Issaquah,"2,183","3,770","10,123","180,902"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,7.2%,19.9%,72.4%
Bellevue,1.3%,8.3%,20.7%,69.7%
Bothell Canyon Park,1.0%,7.2%,11.6%,80.2%
Bremerton,0.4%,3.5%,14.0%,82.1%
Burien,0.3%,2.3%,11.5%,85.9%
Everett,0.0%,8.5%,20.3%,71.2%
Federal Way,0.3%,4.1%,14.9%,80.8%
Greater Downtown Kirkland,0.5%,5.5%,11.6%,82.5%
Issaquah,1.1%,1.9%,5.1%,91.8%


#### AM Peak (7 to 8 AM)

In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,26,400,"1,344","1,808"
Bellevue,64,406,"1,179","4,282"
Bothell Canyon Park,104,"1,529","2,883","6,393"
Bremerton,2,55,159,"1,058"
Burien,16,172,427,"4,182"
Everett,0,653,"1,450","4,267"
Federal Way,4,5,276,"1,068"
Greater Downtown Kirkland,43,"1,984","1,278","6,497"
Issaquah,332,247,"2,290","12,058"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.7%,11.2%,37.6%,50.5%
Bellevue,1.1%,6.8%,19.9%,72.2%
Bothell Canyon Park,1.0%,14.0%,26.4%,58.6%
Bremerton,0.1%,4.3%,12.5%,83.1%
Burien,0.3%,3.6%,8.9%,87.2%
Everett,0.0%,10.2%,22.8%,67.0%
Federal Way,0.3%,0.4%,20.4%,78.9%
Greater Downtown Kirkland,0.4%,20.2%,13.0%,66.3%
Issaquah,2.2%,1.7%,15.3%,80.8%


#### PM Peak (5 to 6 PM)

In [12]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,21,372,784,"2,988"
Bellevue,109,903,"1,613","5,110"
Bothell Canyon Park,255,"1,184","2,645","8,445"
Bremerton,9,55,223,"1,240"
Burien,18,187,891,"5,430"
Everett,0,817,"1,407","5,552"
Federal Way,5,106,265,"1,424"
Greater Downtown Kirkland,65,820,"2,511","8,287"
Issaquah,341,220,"1,025","14,914"


In [13]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,8.9%,18.8%,71.7%
Bellevue,1.4%,11.7%,20.8%,66.1%
Bothell Canyon Park,2.0%,9.5%,21.1%,67.4%
Bremerton,0.6%,3.6%,14.6%,81.2%
Burien,0.3%,2.9%,13.6%,83.2%
Everett,0.0%,10.5%,18.1%,71.4%
Federal Way,0.3%,5.9%,14.7%,79.1%
Greater Downtown Kirkland,0.6%,7.0%,21.5%,70.9%
Issaquah,2.1%,1.3%,6.2%,90.4%


# Network Delay
Totals scaled to match network totals. 

In [14]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
Regional Growth Center,
Auburn,156
Bellevue,301
Bothell Canyon Park,806
Bremerton,18
Burien,82
Everett,165
Federal Way,23
Greater Downtown Kirkland,503
Issaquah,583


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
Regional Growth Center,
Auburn,148
Bellevue,286
Bothell Canyon Park,764
Bremerton,17
Burien,78
Everett,157
Federal Way,22
Greater Downtown Kirkland,477
Issaquah,552
